In [33]:
import cv2
import mediapipe as mp
import time
import math

class HandDetector:
    
    def __init__(self,mode = False,maxHands = 2, complexity = 1, detectionCon = 0.5,trackCon = 0.5):
        
        self.mode = mode
        self.maxHands = maxHands
        self.complexity = complexity
        self.detectionCon = detectionCon
        self.trackCon = trackCon
        
        self.mpDraw = mp.solutions.drawing_utils
        self.mpHand = mp.solutions.hands
        
        self.Hands = self.mpHand.Hands(self.mode,self.maxHands,self.complexity,
                                       self.detectionCon,self.trackCon)
        
        self.tipIds = [4,8,12,16,20]
        
    def findHands(self,image, draw = True,flipType = True):
        
        imageRGB = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        self.results = self.Hands.process(imageRGB)
        allHands = []
        h,w,c = image.shape
        
        if self.results.multi_hand_landmarks:
            for handType,handLms in zip(self.results.multi_handedness,self.results.multi_hand_landmarks):
                myHand = {}
                mylmList = []
                xList = []
                yList = []
                
                for id,lm in enumerate(handLms.landmark):
                    px,py,pz = int(lm.x*w),int(lm.y*h),int(lm.z*w)
                    mylmList.append([px,py,pz])
                    xList.append(px)
                    yList.append(py)
                    
                xmin,xmax = min(xList),max(xList) 
                ymin,ymax = min(yList),max(yList)
                
                bboxW,bboxH = xmax-xmin,ymax-ymin
                bbox = xmin,ymin,xmax,ymax
                
                cx,cy = bbox[0] + (bboxW // 2),bbox[1] + (bboxH // 2)
                
                myHand["lmList"] = mylmList
                myHand["bbox"] = bbox
                myHand["center"] = (cx,cy)
                
                if flipType:
                    if handType.classification[0].label == "Right":
                        myHand["type"] = "Left"
                    else:
                        myHand["type"] = "Right"
                        
                else:
                    
                    myHand["type"] = handType.classification[0].label
                    
                allHands.append(myHand)
                
                if draw:
                    
                    self.mpDraw.draw_landmarks(image,handLms,self.mpHand.HAND_CONNECTIONS)
                    
                    cv2.rectangle(image,(bbox[0]-20,bbox[1] - 20),(bbox[2]+20,bbox[3] + 20),
                                 (0,255,0),2)
                    
                    cv2.putText(image,myHand["type"],(bbox[0]-30,bbox[1] -30),cv2.FONT_HERSHEY_PLAIN,
                               2,(0,255,0),2)
                    
        if draw:
            return allHands,image
        
        else:
            return allHands
        
        
    def fingersUp(self,myHand,flipType=True):
            
        myHandType = myHand["type"]
        mylmList = myHand["lmList"]
        fingers = []
        
        if flipType :
            if myHandType == "Right":

                if mylmList[self.tipIds[0]][0] > mylmList[self.tipIds[0] - 1][0]:
                    fingers.append(1)
                else:
                    fingers.append(0)

            else:

                if mylmList[self.tipIds[0]][0] < mylmList[self.tipIds[0] - 1][0]:
                    fingers.append(1)
                else:
                    fingers.append(0)
                    
        else:
            if myHandType == "Right":

                if mylmList[self.tipIds[0]][0] < mylmList[self.tipIds[0] - 1][0]:
                    fingers.append(1)
                else:
                    fingers.append(0)

            else:

                if mylmList[self.tipIds[0]][0] > mylmList[self.tipIds[0] - 1][0]:
                    fingers.append(1)
                else:
                    fingers.append(0)
                    
            

        for id in range(1,5):
            if mylmList[self.tipIds[id]][1] < mylmList[self.tipIds[id] - 2][1]:
                fingers.append(1)
            else:
                fingers.append(0)

        return fingers
        
        
    def findDistance(self,p1,p2,image = None):
            
        x1,y1 = p1
        x2,y2 = p2
        cx,cy =( x1 + x2 ) // 2,(y1 + y2) // 2
        length = math.hypot(x2-x1,y2-y1)
        info = (x1,y1,x2,y2,cx,cy)

        if image is not None:

            cv2.circle(image, (x1,y1),15,(0,255,0),cv2.FILLED)
            cv2.circle(image, (x2,y2),15,(0,255,0),cv2.FILLED)
            cv2.line(image, (x1,y1),(x2,y2),(0,255,0),3)
            cv2.circle(image, (cx,cy),15,(0,255,0),cv2.FILLED)
            return length,info,image

        return length,info

In [64]:
#!pip install pynput

In [140]:
import cv2
import numpy as np
import time
from pynput.keyboard import Controller

cap = cv2.VideoCapture(0)
cap.set(3,1280)
cap.set(4,720)

hand_detector = HandDetector(detectionCon = 0.85,maxHands=1)

finalText = []
keyboard = Controller()
keys = [["Q", "W", "E", "R", "T", "Y", "U", "I", "O", "P"],
        ["A", "S", "D", "F", "G", "H", "J", "K", "L", ";"],
        ["Z", "X", "C", "V", "B", "N", "M", ",", ".", "/"],
        ["<","_"]]

pTime = 0
def fps():
    global pTime
    cTime = time.time()
    fps = 1 / (cTime - pTime)
    pTime = cTime
    return fps

class Button:
    def __init__(self,pos,text,size = [85,85]):
        
        self.pos = pos
        self.size = size
        self.text = text

def drawALL(image,buttonList):
    
    for button in buttonList:
        
        x,y = button.pos
        w,h = button.size
        CornerLine(image,x,y,w,h)
        cv2.rectangle(image,button.pos,(x+w,y+h),(0,0,255),cv2.FILLED)
        cv2.putText(image,button.text,(x+20,y+65),cv2.FONT_HERSHEY_PLAIN,4,(255,255,255),4)
        
    return image

def CornerLine(image,x,y,w,h,length=20,tickness = 3):
    
    cv2.line(image,(x,y),(x,y+length),(0,255,0),tickness)
    cv2.line(image,(x,y),(x+length,y),(0,255,0),tickness)
    
    cv2.line(image,(x+w,y+h),(x+w,y+h-length),(0,255,0),tickness)
    cv2.line(image,(x+w,y+h),(x+w-length,y+h),(0,255,0),tickness)
    
    cv2.line(image,(x+w,y),(x+w-length,y),(0,255,0),tickness)
    cv2.line(image,(x+w,y),(x+w,y+length),(0,255,0),tickness)
    
    cv2.line(image,(x,y+h),(x,y+h-length),(0,255,0),tickness)
    cv2.line(image,(x,y+h),(x+length,y+h),(0,255,0),tickness)
    
def LastDrawALL(image,buttonList):
    
    imgNew = np.zeros_like(image,np.uint8)
    
    for button in buttonList:
        x,y = button.pos
        w,h = button.size
        
        CornerLine(imgNew,x,y,w,h)
        
        cv2.rectangle(imgNew,button.pos,(x+w,y+h),(0,0,255),cv2.FILLED)
        cv2.putText(imgNew,button.text,(x+40,y+60),cv2.FONT_HERSHEY_PLAIN,2,(255,255,255),3)
        
    out = image.copy()
    alpha = 0.5
    mask = imgNew.astype(bool)
    out[mask] = cv2.addWeighted(image,alpha,imgNew ,1-alpha,0)[mask]
    
    return out

buttonList = []
for i in range(len(keys)):
    for j,key in enumerate(keys[i]):
        buttonList.append(Button([100*j+70,100 * i + 100], key))
            
while True:
    
    conn,frame = cap.read()
    frame = cv2.flip(frame,1)

    #frame = drawALL(frame,buttonList)
    frame = LastDrawALL(frame,buttonList)
    
    allhands,frame = hand_detector.findHands(frame,draw = True,flipType = False)
    
    if allhands:
        hand1 = allhands[0]
        type1 = hand1["type"]
        center1 = hand1["center"]
        bbox1 = hand1["bbox"]
        lmList1 = hand1["lmList"]
    
        for button in buttonList:
            x,y = button.pos
            w,h = button.size
            
            if x < lmList1[8][0] < x+w and y < lmList1[8][1] < y + h:
                
                cv2.rectangle(frame,button.pos,(x+w,y+h),(0,0,125),cv2.FILLED)
                cv2.putText(frame,button.text,(x+20,y+65),cv2.FONT_HERSHEY_PLAIN,4,(255,255,255),4)
                length,info,frame = hand_detector.findDistance(lmList1[8][:-1],lmList1[12][:-1],frame)
                
                # When Clicked
                if length < 35:
                    if button.text == "<":

                        try:  
                            cv2.rectangle(frame,button.pos,(x+w,y+h),(0,255,0),cv2.FILLED)
                            cv2.putText(frame,button.text,(x+20,y+65),cv2.FONT_HERSHEY_PLAIN,4,(255,255,255),4)
                            finalText.pop(-1) 
                        except:
                            pass

                    elif button.text == "_":
                        cv2.rectangle(frame,button.pos,(x+w,y+h),(0,255,0),cv2.FILLED)
                        cv2.putText(frame,button.text,(x+20,y+65),cv2.FONT_HERSHEY_PLAIN,4,(255,255,255),4)
                        finalText.append(" ")

                    else:

                        keyboard.press(button.text)
                        cv2.rectangle(frame,button.pos,(x+w,y+h),(0,255,0),cv2.FILLED)
                        cv2.putText(frame,button.text,(x+20,y+65),cv2.FONT_HERSHEY_PLAIN,4,(255,255,255),4)

                        finalText.append(button.text)
                        time.sleep(0.3)
                                      
    cv2.rectangle(frame,(60,500),(700,600),(0,0,255),cv2.FILLED)
    cv2.putText(frame,"".join(finalText),(70,580),cv2.FONT_HERSHEY_PLAIN,4,(255,255,255),4)                
                    
    fps_ = fps()
    cv2.putText(frame,f"FPS : {int(fps_)}",(20,70),cv2.FONT_HERSHEY_PLAIN,3,(255,0,0),3)
    cv2.imshow("Frame",frame)
    if cv2.waitKey(1) & 0xff == ord("q"):
        break
        
        
cap.release()    
cv2.destroyAllWindows()

## 

In [177]:
import cv2
import numpy as np


cap = cv2.VideoCapture(0)
cap.set(3,1280)
cap.set(4,720)

hand_detector = HandDetector(maxHands=1)

class Button:
    def __init__(self,pos,width,height,value):
        
        self.pos = pos    
        self.width = width
        self.height = height
        self.value = value
        
    def draw(self,image):

        cv2.rectangle(image,self.pos,(self.pos[0] + self.width, self.pos[1] + self.height),
                      (225,225,225),cv2.FILLED)
        cv2.rectangle(image,self.pos,(self.pos[0] + self.width, self.pos[1] + self.height),(50,50,50),3)
        cv2.putText(image,self.value,(self.pos[0] + 40,self.pos[1] + 60),cv2.FONT_HERSHEY_PLAIN,2,(50,50,50),2)
        
    def check_click(self,image,x,y):
    
        if self.pos[0] < x < self.pos[0] + self.width and \
            self.pos[1] < y < self.pos[1] + self.height:
            
            cv2.rectangle(image,self.pos,(self.pos[0] + self.width, self.pos[1] + self.height),
                          (225,225,225),cv2.FILLED)
            cv2.rectangle(image,self.pos,(self.pos[0] + self.width, self.pos[1] + self.height),(50,50,50),3)
            cv2.putText(image,self.value,(self.pos[0] + 40,self.pos[1] + 60),cv2.FONT_HERSHEY_PLAIN,5,(0,0,0),5)
            
            return True
        
        else:
            return False
          
button1 = Button((700,150),100,100,"5")
ButtonListValues = [['7','8','9','*'],
                    ['4','5','6','-'],
                    ['1','2','3','+'],
                    ['0','/','<','=']]

buttonList = []
for x in range(4):
    for y in range(4):
        xpos = x*100 + 800
        ypos = y*100 + 150
        buttonList.append(Button((xpos,ypos),100,100,ButtonListValues[y][x]))
    
myEquation = ""   
delayCounter = 0

while True:
    
    conn,frame = cap.read()
    frame = cv2.flip(frame,1)
    
    allHands,frame = hand_detector.findHands(frame,draw = True,flipType=False)
    
    #button1.draw(frame)
    
    ## Yukardaki hesaplama ya göre düşün !!!
    cv2.rectangle(frame,(800,50),(800+400,70+100),(225,225,225),cv2.FILLED)
    cv2.rectangle(frame,(800,50),(800+400,70+100),(50,50,50),3)
    
    for button in buttonList:
        button.draw(frame)
    
    # Processing
    if allHands:
        hands1 = allHands[0]
        lmList1 = hands1["lmList"]
        
        lenght,info,frame = hand_detector.findDistance(lmList1[8][:-1],lmList1[12][:-1],frame)
        #print(lenght)
        x,y = lmList1[8][:-1]
        if lenght < 50:
            for i,button in enumerate(buttonList):
                if button.check_click(frame,x,y) and delayCounter == 0:
                    # Biraz düşündürücü !!!
                    #print(ButtonListValues[int(i%4)][int(i/4)])
                    #myValue = ButtonListValues[int(i%4)][int(i/4)]
                    
                    # Basic method :D
                    myValue = button.value
                    #time.sleep(.3)
                    
                    if myValue == "<":
                        myEquation = myEquation[:-1]
                    
                    elif myValue == "=":
                        myEquation = str(eval(myEquation))
                        
                    else:
                        myEquation+=myValue
                    delayCounter+=1
     
    # Lag Avoid !!
    if delayCounter != 0:
        delayCounter+=1
        if delayCounter > 10:
            delayCounter = 0
    
    # Display The Results/Equalitons
    cv2.putText(frame,myEquation,(810,110),cv2.FONT_HERSHEY_PLAIN,2,(50,50,50),2)
    cv2.imshow("Frame",frame)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break
           
cap.release()    
cv2.destroyAllWindows()

In [149]:
ButtonListValues

[['7', '8', '9', '*'],
 ['4', '5', '6', '-'],
 ['1', '2', '3', '+'],
 ['0', '/', '.', '=']]

In [155]:
l = len(buttonList)
l

16

In [156]:
# diyelim ben 8. listedeki yere denk geldim !!!
int(8 % 4), int(8 / 4)

(0, 2.0)

In [158]:
buttonList[8].value,ButtonListValues[0][2]

('9', '9')

In [165]:
int(9%4),int(9/4)

(1, 2)

In [166]:
buttonList[9].value,ButtonListValues[1][2]

('6', '6')

In [168]:
eval("5*3")

15